
    This Source Code Form is subject to the terms of the Mozilla Public
    License, v. 2.0. If a copy of the MPL was not distributed with this
    file, You can obtain one at http://mozilla.org/MPL/2.0/.


Please be sure that all modules listed below are installed on your machine.

In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.transforms
import numpy as np
import pandas as pd
import seaborn as sns
from statsmodels.stats import weightstats as stests
#Violinplot à tester

### Global parameter to build seaborn boxplot

In [ ]:
# If necessary, update the path where all data are located.
# All computed p-values are gathered in a CSV file saved in the folder 'path'
# whose file name depends on the experiment results you want to see
path = 'data/TPNC/'
offset_path = len(path)

# Comment or uncomment the following lines to see results (figures and p-values) for each conducted experiment
# Source code will be available on github soon...

# 0 ) PEPACS vs EPEACS vs ACS2
#     On all mazes without slippery and random attributes
#file_name_lcs_1 = 'acs2.json'
#file_name_lcs_1 = 'pepacs.json'

# A ) PEPACS vs EPEACS
#     On all mazes with random attribute with fifty per cent chance to get each random attribute
#file_name_lcs_1 = 'acs2_ra_50_50.json'
#file_name_lcs_2 = 'pepacs_ra_50_50.json'

# B ) PEPACS vs EPEACS
#     On all mazes with random attribute with 90_10 per cent chance to get each random attribute
#file_name_lcs_1 = 'acs2_ra_90_10.json'
#file_name_lcs_2 = 'pepacs_ra_90_10.json'

# C ) PEPACS vs EPEACS
#     On all mazes with slippery
#file_name_lcs_1 = 'acs2_slippery.json'
#file_name_lcs_2 = 'pepacs_slippery.json'

# D ) PEPACS vs EPEACS
#     On all mazes with slippery and random attribute
#file_name_lcs_1 = 'acs2_slippery_ra.json'
#file_name_lcs_2 = 'pepacs_slippery_ra.json'


# E ) BACS vs BACASABLE
#     On all mazes without slippery, ra, ga and with bseq =1 and pep
#file_name_lcs_1 = '../bacs-bseq2-1000-0.8-ga-anticipatechange.json'
#file_name_lcs_1 = '../bacs-bseq2-1000-0.8-ga-gaonbseq-anticipatechange.json'
#file_name_lcs_1 = '../bacs-bseq2-1000-0.8.json'
#file_name_lcs_1 = '../bacs-bseq2-1000-0.8-ga.json'
#file_name_lcs_1 = '../bacs-bseq2-5000-0.8.json'
#file_name_lcs_1 = '../bacs-bseq2-5000-0.8-ga.json'
#file_name_lcs_2 = '../epeacs-bseq2-5000-0.8.json'
#file_name_lcs_2 = '../epeacs-bseq2-5000-0.8-ga.json'
#file_name_lcs_2 = '../epeacs-bseq2-5000-0.8-slippery-0.25.json'
#file_name_lcs_2 = '../epeacs-bseq2-5000-0.8-ga-slippery-0.25.json'


# Font size et figuresize parameters for plotting
figure_size = (20, 10)
axe_label_fontsize = '28'
tick_label_fontsize = '24'
legend_fontsize = '24'

# Significance level for p-values
alpha = 0.05
produce_stat = False

### Please, do not modify any lines of code below

### Reading json files

In [ ]:
def read_json(path, file_name_lcs_1, file_name_lcs_2):
    json_data_from_lcs_1 = path + file_name_lcs_1
    json_data_from_lcs_2 = path + file_name_lcs_2

    with open(json_data_from_lcs_1) as json_file_1:
        raw_data_from_lcs_1 = json.load(json_file_1)

    with open(json_data_from_lcs_2) as json_file_2:
        raw_data_from_lcs_2 = json.load(json_file_2)

    raw_data = [raw_data_from_lcs_1, raw_data_from_lcs_2, raw_data_from_lcs_2]
    lcs_name = [file_name_lcs_1, file_name_lcs_2]
    #lcs_name = ['ACS2', 'PEPACS', 'EPEACS']
    return raw_data, lcs_name

# Call function to prepare plotting data
raw_data, lcs_name = read_json(path, file_name_lcs_1, file_name_lcs_2)

### Preparing pandas plotting data

In [ ]:
def prepare_pandas_plotting_data(raw_data, lcs_name):
    cleaned_data = {
        'LCS':[], 
        'Maze':[], 
        'Knowledge (%)':[],
        'Population of classifiers':[], 
        'Reliable classifiers':[], 
        'Mean realiable classifier specificity':[], 
        #'Full Knowledge for first time (trial)':[],
        #'Full Knowledge stable (trial)':[],
        #'Full Knowledge for last time (trial)':[],
        'PEP Accumulated Error (%)':[],
        'Average steps to exit':[]
    }
    nb_of_environments = 0
    for i in range(len(lcs_name)):
        for item in raw_data[i]:
            if 'time' not in item.keys():
                nb_of_environments += 1
                for idx in range(len(item['knowledge_list'])):
                    cleaned_data['LCS'].append(lcs_name[i])
                    if item['maze'][:-3] == "Woods101demi":
                        cleaned_data['Maze'].append("Woods101.5")
                    else:
                        cleaned_data['Maze'].append(item['maze'][:-3])
                    cleaned_data['Knowledge (%)'].append(item['knowledge_list'][idx])
                    cleaned_data['Population of classifiers'].append(item['population_list'][idx])
                    cleaned_data['Reliable classifiers'].append(item['reliable_list'][idx])
                    cleaned_data['Mean realiable classifier specificity'].append(
                        item['mean_reliable_classifier_specificity_list'][idx]
                    )
                    #cleaned_data['Full Knowledge for first time (trial)'].append(
                    #    item['full_knowledge_first_trial_list'][idx]
                    #)
                    #cleaned_data['Full Knowledge stable (trial)'].append(
                    #    item['full_knowledge_stable_trial_list'][idx]
                    #)
                    #cleaned_data['Full Knowledge for last time (trial)'].append(
                    #    item['full_knowledge_last_trial_list'][idx]
                    #)
                    if i == 1:
                        cleaned_data['PEP Accumulated Error (%)'].append(item['old_pep_error_list'][idx])
                    else:
                        cleaned_data['PEP Accumulated Error (%)'].append(item['new_pep_error_list'][idx])
                    cleaned_data['Average steps to exit'].append(
                        item['avg_exploit_rl_list'][idx]
                    )

    nb_of_environments /= len(lcs_name)
    pandas_data = pd.DataFrame(cleaned_data)
    return nb_of_environments, pandas_data

nb_of_environments, pandas_data = prepare_pandas_plotting_data(raw_data, lcs_name)

### Plotting boxplots with seaborn

In [ ]:
rc={
    'axes.labelsize':axe_label_fontsize,
    'xtick.labelsize':tick_label_fontsize,
    'ytick.labelsize':tick_label_fontsize
}
sns.set(context='notebook', style='ticks', rc=rc)

In [ ]:
def plot_figure(pandas_data, Y, legend_loc, lcs_name):
    
    # Build the main figure
    fig, ax = plt.subplots(figsize=figure_size)
    sns.stripplot(
        x = 'Maze',
        y = Y,
        hue = 'LCS',
        data = pandas_data,
        edgecolor='gray',
        jitter=True,
        linewidth = 1,
        dodge=True,
        palette='pastel',
        ax = ax
    )#violinplot
    sns.boxplot(
        x = 'Maze',
        y = Y,
        hue = 'LCS',
        data = pandas_data,
        fliersize=0,
        palette='pastel',
        ax = ax
    )

    # Set up x tick labels and x label correctly
    plt.xticks(rotation = 45, horizontalalignment = 'right')
    plt.xlabel('')

    # Hide the horizontal gridlines
    ax.yaxis.grid(False)

    # Show the vertical gridlines
    ax.xaxis.grid(True)

    # Shift major ticks to create column for each maze
    plt.xticks(np.arange(nb_of_environments)+0.5)

    # Set up minor ticks and align label of major ticks on minor ticks
    ax.xaxis.set_minor_locator(plt.MultipleLocator(0.5))
    dx = (((figure_size[0] * 100 / 4.))/nb_of_environments - 10)/72.; dy = 0/72. # Width in inches converted into pixels depending on default DPI
    offset = matplotlib.transforms.ScaledTranslation(dx, dy, fig.dpi_scale_trans)
    for label in ax.xaxis.get_majorticklabels():
        label.set_transform(label.get_transform() - offset)

    # Set up legend
    handles, labels = ax.get_legend_handles_labels()
    if len(lcs_name) == 2:
        plt.legend(handles[0:2], labels[0:2], frameon=False, loc=legend_loc, fontsize=legend_fontsize)
    else:
        plt.legend(handles[0:3], labels[0:3], frameon=False, loc=legend_loc, fontsize=legend_fontsize)

In [ ]:
def plot_knowledge(pandas_data, zipped:bool=True):
    plot_figure(pandas_data, Y='Knowledge (%)', legend_loc='lower right', lcs_name=lcs_name)
    
def plot_population(pandas_data, only_reliable:bool=False):
    if not only_reliable:
        Y = 'Population of classifiers'
    else:
        Y = 'Reliable classifiers'
    plot_figure(pandas_data, Y=Y, legend_loc='upper right', lcs_name=lcs_name)

def plot_mean_classifier_specificity(pandas_data):
    plot_figure(pandas_data, Y='Mean realiable classifier specificity', legend_loc='upper right', lcs_name=lcs_name)

#def plot_first_time_full_knowledge_achieved(pandas_data):
#    plot_figure(pandas_data, Y='Full Knowledge for first time (trial)', legend_loc='upper right', lcs_name=lcs_name)

#def plot_last_time_full_knowledge_achieved(pandas_data):
#    plot_figure(pandas_data, Y='Full Knowledge for last time (trial)', legend_loc='lower right', lcs_name=lcs_name)

def plot_pep_accumulated_error(pandas_data):
    plot_figure(pandas_data, Y='PEP Accumulated Error (%)', legend_loc='upper right', lcs_name=lcs_name)

def plot_avg_steps_to_exit(pandas_data):
    plot_figure(pandas_data, Y='Average steps to exit', legend_loc='upper right', lcs_name=lcs_name)

In [ ]:
plot_knowledge(pandas_data)

In [ ]:
plot_population(pandas_data, only_reliable=False)
plot_population(pandas_data, only_reliable=True)

In [ ]:
plot_mean_classifier_specificity(pandas_data)

In [ ]:
plot_pep_accumulated_error(pandas_data)

In [ ]:
plot_avg_steps_to_exit(pandas_data)

In [ ]:
#plot_first_time_full_knowledge_achieved(pandas_data)
#plot_last_time_full_knowledge_achieved(pandas_data)

### Preparing statistical data from raw read json

In [ ]:
if produce_stat:
    raw_list_by_env_name = {}
    for i in range(len(lcs_name)):
        for item in raw_data[i]:
            if 'time' not in item.keys():
                if item['maze'] not in raw_list_by_env_name.keys():
                    raw_list_by_env_name[item['maze']] = {
                        'knowledge_list' : [],
                        'population_list' : [],
                        'reliable_list' : [],
                        'mean_reliable_classifier_specificity_list' : [],
                        'full_knowledge_first_trial_list' : [],
                        'full_knowledge_last_trial_list' : [],
                        'PEP_Accumulated_Error' : []
                    }
                raw_list_by_env_name[item['maze']]['knowledge_list'].append(
                    np.array(item['knowledge_list']))
                raw_list_by_env_name[item['maze']]['population_list'].append(
                    np.array(item['population_list']))
                raw_list_by_env_name[item['maze']]['reliable_list'].append(
                    np.array(item['reliable_list']))
                raw_list_by_env_name[item['maze']]['mean_reliable_classifier_specificity_list'].append(
                    np.array(item['mean_reliable_classifier_specificity_list']))
                raw_list_by_env_name[item['maze']]['full_knowledge_first_trial_list'].append(
                    np.array(item['full_knowledge_first_trial_list']))
                raw_list_by_env_name[item['maze']]['full_knowledge_last_trial_list'].append(
                    np.array(item['full_knowledge_last_trial_list']))
                if i == 1:
                    raw_list_by_env_name[item['maze']]['PEP_Accumulated_Error'].append(
                        np.array(item['old_pep_error_list']))
                else:
                    raw_list_by_env_name[item['maze']]['PEP_Accumulated_Error'].append(
                        np.array(item['new_pep_error_list']))

### Computing p-values

In [ ]:
if produce_stat:
    raw_statistical_data = {
        'Maze':[],
        'Metric':[],
        'ALCS':[],
        'Alternative':[],
        'Degrees of freedom':[],
        'P Value':[],
        'Null Hypothesis':[]
    }
    for env_name in raw_list_by_env_name.keys():
        for metric in raw_list_by_env_name[env_name].keys():
            for i in range(3):
                raw_statistical_data['Maze'].append(env_name)
                raw_statistical_data['Metric'].append(metric)
                raw_statistical_data['ALCS'].append('ACS2-PEPACS')
                x1 = np.array(raw_list_by_env_name[env_name][metric][0])
                x2 = np.array(raw_list_by_env_name[env_name][metric][1])
                if i == 0:
                    alt = 'two-sided'
                elif i == 1:
                    alt = 'larger'
                else:
                    alt = 'smaller'
                raw_statistical_data['Alternative'].append(alt)
                tstats, pvalue, dof = stests.ttest_ind(x1, x2, alternative=alt, usevar='unequal', value=0)
                raw_statistical_data['Degrees of freedom'].append(dof)
                raw_statistical_data['P Value'].append(pvalue)
                if pvalue<alpha:
                    raw_statistical_data['Null Hypothesis'].append('Reject')
                else:
                    raw_statistical_data['Null Hypothesis'].append('Accept')

                raw_statistical_data['Maze'].append(env_name)
                raw_statistical_data['Metric'].append(metric)
                raw_statistical_data['ALCS'].append('ACS2-EPEACS')
                x1 = np.array(raw_list_by_env_name[env_name][metric][0])
                x2 = np.array(raw_list_by_env_name[env_name][metric][2])
                if i == 0:
                    alt = 'two-sided'
                elif i == 1:
                    alt = 'larger'
                else:
                    alt = 'smaller'
                raw_statistical_data['Alternative'].append(alt)
                tstats, pvalue, dof = stests.ttest_ind(x1, x2, alternative=alt, usevar='unequal', value=0)
                raw_statistical_data['Degrees of freedom'].append(dof)
                raw_statistical_data['P Value'].append(pvalue)
                if pvalue<alpha:
                    raw_statistical_data['Null Hypothesis'].append('Reject')
                else:
                    raw_statistical_data['Null Hypothesis'].append('Accept')

                raw_statistical_data['Maze'].append(env_name)
                raw_statistical_data['Metric'].append(metric)
                raw_statistical_data['ALCS'].append('PEPACS-EPEACS')
                x1 = np.array(raw_list_by_env_name[env_name][metric][1])
                x2 = np.array(raw_list_by_env_name[env_name][metric][2])
                if i == 0:
                    alt = 'two-sided'
                elif i == 1:
                    alt = 'larger'
                else:
                    alt = 'smaller'
                raw_statistical_data['Alternative'].append(alt)
                tstats, pvalue, dof = stests.ttest_ind(x1, x2, alternative=alt, usevar='unequal', value=0)
                raw_statistical_data['Degrees of freedom'].append(dof)
                raw_statistical_data['P Value'].append(pvalue)
                if pvalue<alpha:
                    raw_statistical_data['Null Hypothesis'].append('Reject')
                else:
                    raw_statistical_data['Null Hypothesis'].append('Accept')
    statistical_pandas = pd.DataFrame(raw_statistical_data)

In [ ]:
if produce_stat:
    statistical_pandas.to_csv(path + file_name_lcs_1 + '_&_' + file_name_lcs_2 + '_stats.csv')